**Importing the libraries**

In [8]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import gc
import numpy as np

from keras import backend as K
from sklearn.metrics import fbeta_score
from keras.layers import Conv2D, Dense, LSTM, Flatten, MaxPooling2D, BatchNormalization, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import fbeta_score
from tqdm import tqdm
from sklearn.utils import shuffle

import cv2
from PIL import Image
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential 

import seaborn as sns
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, History

**Loading the datasets**

In [9]:
#reading the labels

train_label = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')
sam_sub = pd.read_csv('../input/planets-dataset/planet/planet/sample_submission.csv')
train_label.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [10]:
#onehot=OneHotEncoder()

encoder = LabelEncoder()

In [11]:
#encoder.fit(train_label['tags'])

label_maps = pd.DataFrame()
label_maps['tags'] = ['agriculture', 'artisinal_mine', 'bare_ground','blooming','blow_down','clear','cloudy','conventional_mine','cultivation','habitation','haze', 'partly_cloudy','primary','road','selective_logging','slash_burn','water']
label_maps['map'] = encoder.fit_transform(label_maps['tags'])
label_maps

,tags,map
0,agriculture,0
1,artisinal_mine,1
2,bare_ground,2
3,blooming,3
4,blow_down,4
5,clear,5
6,cloudy,6
7,conventional_mine,7
8,cultivation,8
9,habitation,9


In [12]:
#defining a dict of encoded labels

label_map = {'agriculture': 0,
 'artisinal_mine': 1,
 'bare_ground': 2,
 'blooming': 3,
 'blow_down': 4,
 'clear': 5,
 'cloudy': 6,
 'conventional_mine': 7,
 'cultivation': 8,
 'habitation': 9,
 'haze': 10,
 'partly_cloudy': 11,
 'primary': 12,
 'road': 13,
 'selective_logging': 14,
 'slash_burn': 15,
 'water': 16}


In [13]:
#loading the traing_images

X = []
Y = []
train_label = shuffle(train_label,random_state=0)

In [14]:
for image_name, tags in tqdm(train_label.values, miniters=400):
    arr = cv2.imread('../input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(image_name), cv2.IMREAD_UNCHANGED)
    targets = np.zeros(17)
    for t in tags.split(' '):
      targets[label_map[t]] = 1 
    arr = cv2.resize(arr, (64, 64))
    X.append(arr)
    Y.append(targets)

100%|██████████| 40479/40479 [06:59<00:00, 96.52it/s]


In [15]:
X = np.array(X, np.float16)/255.0

In [16]:
#splitting

X = np.array(X)
Y = np.array(Y)
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size = 0.1, shuffle = True, random_state = 69)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(36431, 64, 64, 3) (36431, 17) (4048, 64, 64, 3) (4048, 17)


In [17]:
del X, Y
gc.collect()

69

In [18]:
gc.collect()

23

In [19]:
def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 2

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

In [20]:
from tensorflow.keras.optimizers import Adam

**Define the model**

In [22]:
model = keras.Sequential()
model.add(Conv2D(64, 5, 2, activation = "relu", input_shape = (64, 64, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(128, 5, 2, activation = "relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dense(17, activation = "sigmoid"))
model.compile(loss = "binary_crossentropy", optimizer = Adam(), metrics = [fbeta])
model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 30, batch_size = 64)

Epoch 1/30
570/570 [==============================] - 37s 63ms/step - loss: 0.2253 - fbeta: 0.6335 - val_loss: 0.2063 - val_fbeta: 0.7023
Epoch 2/30
570/570 [==============================] - 35s 62ms/step - loss: 0.1977 - fbeta: 0.6832 - val_loss: 0.1867 - val_fbeta: 0.6954
Epoch 3/30
570/570 [==============================] - 36s 63ms/step - loss: 0.1834 - fbeta: 0.7117 - val_loss: 0.1751 - val_fbeta: 0.7449
Epoch 4/30
570/570 [==============================] - 35s 62ms/step - loss: 0.1675 - fbeta: 0.7419 - val_loss: 0.1640 - val_fbeta: 0.7527
Epoch 5/30
570/570 [==============================] - 35s 62ms/step - loss: 0.1585 - fbeta: 0.7561 - val_loss: 0.1553 - val_fbeta: 0.7749
Epoch 6/30
570/570 [==============================] - 35s 62ms/step - loss: 0.1529 - fbeta: 0.7659 - val_loss: 0.1490 - val_fbeta: 0.7712
Epoch 7/30
570/570 [==============================] - 35s 62ms/step - loss: 0.1494 - fbeta: 0.7719 - val_loss: 0.1549 - val_fbeta: 0.7826
Epoch 8/30
570/570 [==============

In [23]:
gc.collect()

1793

In [24]:
#with tpu_strategy.scope():

test_loss, test_accuracy = model.evaluate(x_val, y_val)
print('Test loss: {}'.format(test_loss))
print('Test accuracy: {}'.format(test_accuracy))

127/127 [==============================] - 1s 10ms/step - loss: 0.1388 - fbeta: 0.8171
Test loss: 0.13877493143081665
Test accuracy: 0.8170564770698547


In [25]:
gc.collect()

840

In [26]:
del x_train, y_train, x_val, y_val
gc.collect()

23

In [27]:
#dividing my test_labels into two part for test-jpg and test-jpg-additional
test = sam_sub[0 : 40669]
files = sam_sub[40669 : ]

In [28]:
#with tpu_strategy.scope():  

test_img = []

for image_name, tags in tqdm(test.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/planet/planet/test-jpg/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

for image_name, tags in tqdm(files.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/test-jpg-additional/test-jpg-additional/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

test_img = np.array(test_img, np.float16)/255.0

100%|██████████| 20522/20522 [03:52<00:00, 88.31it/s] 


In [29]:
gc.collect()

69

In [30]:
#with tpu_strategy.scope():

yres = []
predictions = model.predict(test_img, batch_size = 64, verbose = 2)
yres.append(predictions)

957/957 - 19s


In [31]:
gc.collect()

783

In [32]:
#converting my encoded labels back to it original form

sub = np.array(yres[0])
for i in range (1, len(yres)):
    sub += np.array(yres[i])
sub = pd.DataFrame(sub, columns = label_map)

In [33]:
sub

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,0.010665,7.496330e-08,2.993941e-04,1.410925e-02,1.327962e-03,9.953507e-01,1.638872e-05,3.329211e-07,0.007939,0.004087,3.665388e-03,0.000427,0.999845,0.023510,5.234301e-03,3.650348e-06,0.051547
1,0.045668,3.493889e-10,1.416206e-04,1.422215e-02,5.929649e-03,9.558641e-01,1.648842e-06,1.242141e-08,0.035020,0.001522,7.110089e-05,0.122598,0.999994,0.028347,1.627925e-02,2.850706e-05,0.095286
2,0.031728,2.621980e-08,3.584947e-06,2.825359e-11,1.796921e-06,5.353323e-07,8.617463e-09,3.869809e-05,0.007173,0.000097,3.207919e-08,0.999986,0.999997,0.004680,3.252461e-09,4.380742e-05,0.057227
3,0.030141,1.689406e-09,2.852976e-04,2.768233e-02,2.340585e-03,6.382115e-01,5.307116e-07,1.025899e-06,0.020302,0.002005,1.288056e-04,0.454712,0.999999,0.008741,6.987542e-03,3.997662e-06,0.025021
4,0.460094,2.841132e-08,1.255035e-03,1.130074e-12,3.154814e-11,1.205131e-04,1.271865e-02,2.778441e-07,0.001309,0.008473,1.005236e-05,0.990269,0.995065,0.265219,4.730447e-08,1.715804e-07,0.069653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61186,0.000058,1.579102e-07,1.496673e-04,7.902276e-10,8.668809e-14,3.356034e-02,7.819186e-01,2.564383e-06,0.000362,0.005274,1.174336e-04,0.003903,0.178307,0.015467,2.707127e-07,2.427350e-09,0.018508
61187,0.101827,5.083273e-08,3.298700e-03,1.188395e-08,1.842441e-12,9.992090e-01,5.373829e-09,2.933443e-04,0.078999,0.006308,1.347322e-05,0.000245,0.999987,0.035206,2.687852e-05,6.928716e-06,0.925841
61188,0.020404,9.707046e-11,2.105236e-04,2.338308e-02,5.184352e-03,9.938282e-01,1.645316e-07,3.357686e-08,0.015994,0.002145,2.582083e-05,0.014174,0.999999,0.019191,1.019767e-02,4.254385e-06,0.044785
61189,0.026465,1.934147e-11,3.172117e-07,2.615024e-11,2.646951e-10,9.071708e-03,6.918700e-01,2.305048e-09,0.002564,0.004028,2.372493e-01,0.000040,0.280314,0.037782,2.080781e-09,1.980445e-10,0.043804


In [34]:
preds = []
for i in tqdm(range(sub.shape[0]), miniters=1000):
    a = sub.loc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a= a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

sam_sub['tags'] = preds
sam_sub.to_csv('subm.csv', index=False)

100%|██████████| 61191/61191 [01:58<00:00, 515.52it/s]
